In [53]:
import numpy as np
import pandas as pd


# import matplotlib
# matplotlib.use('WXAgg')  # Replace 'TkAgg' with your preferred backend
import matplotlib.pyplot as plt

# import matplotlib
# # be able to save images on server
# matplotlib.use('Agg')
# from matplotlib import pyplot


from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.datasets import cifar10
from keras.utils import normalize, to_categorical
import time
from numpy.random import randn
from numpy.random import randint


import keras

from keras.callbacks import LearningRateScheduler

from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
import keras_tuner as kt
from keras.wrappers.scikit_learn import KerasClassifier
# define the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [54]:
# Input the data
data = pd.read_csv('marbledGodwit_USGS_ASC_argos_1.csv');

b=data.shape;   #shape
# print(b)

In [55]:
# filter out one animols
animal_ID_MAGO_7H = data[data['individual-local-identifier']=='MAGO_7H']
# animal_ID_MAGO_7H

In [56]:
import time
import datetime
# timestamp
animal_ID_MAGO_7H.timestamp = pd.to_datetime(animal_ID_MAGO_7H.timestamp)
animal_ID_MAGO_7H.head()

C:\Users\28595\anaconda3\envs\SNS\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,event-id,visible,timestamp,location_long,location_lat,algorithm-marked-outlier,argos:altitude,argos:best-level,argos:calcul-freq,argos:error-radius,...,argos:valid-location-algorithm,argos:valid-location-manual,comments,manually-marked-outlier,manually-marked-valid,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
5154,6908808897,True,2008-06-18 05:09:00,-157.611,57.443,NaN,0.016,-127,402000000.0,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
5155,6908808898,True,2008-06-18 06:12:00,-157.659,57.402,NaN,0.017,-129,402000000.0,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
5156,6908808900,True,2008-06-18 07:52:00,-157.686,57.458,NaN,0.023,-124,402000000.0,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
5157,6908808901,True,2008-06-18 08:31:00,-157.637,57.477,NaN,0.019,-121,402000000.0,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
5158,6908808902,True,2008-06-18 09:32:00,-157.638,57.453,NaN,0.017,-131,402000000.0,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos


In [57]:
# set index (datetime)
animal_ID_MAGO_7H = animal_ID_MAGO_7H.set_index('timestamp')
animal_ID_MAGO_7H.head()

# sort the index
animal_ID_MAGO_7H.sort_index(ascending = True).head()

,event-id,visible,location_long,location_lat,algorithm-marked-outlier,argos:altitude,argos:best-level,argos:calcul-freq,argos:error-radius,argos:gdop,...,argos:valid-location-algorithm,argos:valid-location-manual,comments,manually-marked-outlier,manually-marked-valid,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
timestamp,,,,,,,,,,,,,,,,,,,,,
2008-06-18 05:09:00,6908808897,True,-157.611,57.443,NaN,0.016,-127,402000000.0,NaN,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
2008-06-18 06:12:00,6908808898,True,-157.659,57.402,NaN,0.017,-129,402000000.0,NaN,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
2008-06-18 07:52:00,6908808900,True,-157.686,57.458,NaN,0.023,-124,402000000.0,NaN,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
2008-06-18 08:31:00,6908808901,True,-157.637,57.477,NaN,0.019,-121,402000000.0,NaN,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos
2008-06-18 09:32:00,6908808902,True,-157.638,57.453,NaN,0.017,-131,402000000.0,NaN,NaN,...,1,NaN,alive,NaN,NaN,argos-doppler-shift,Limosa fedoa,80783,MAGO_7H,marbledGodwit_USGS_ASC_argos


In [58]:
# the location of animal
animal_ID_MAGO_7H_pre_data = animal_ID_MAGO_7H[animal_ID_MAGO_7H.columns[2:4]]
animal_ID_MAGO_7H_pre_data

a = animal_ID_MAGO_7H_pre_data.resample('D').mean()
animal_ID_MAGO_7H_data = a.resample('D').interpolate('linear')

animal_ID_MAGO_7H_data

,location_long,location_lat
timestamp,,
2008-06-18,-157.643429,57.450143
2008-06-19,-157.660214,57.430929
2008-06-20,-157.677000,57.411714
2008-06-21,-157.663125,57.430732
2008-06-22,-157.649250,57.449750
...,...,...
2012-05-23,-157.621167,57.501667
2012-05-24,-157.626512,57.515190
2012-05-25,-157.631857,57.528714


In [59]:
pre_days =10
animal_ID_MAGO_7H_data['label_1'] = animal_ID_MAGO_7H_data['location_long'].shift(-pre_days)
animal_ID_MAGO_7H_data['label_2'] = animal_ID_MAGO_7H_data['location_lat'].shift(-pre_days)
# Standardlise
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
data_scalar = scalar.fit_transform(animal_ID_MAGO_7H_data.iloc[:,:-2])
print(data_scalar)

[[-1.91242768  1.80728433]
 [-1.91357816  1.80506405]
 [-1.91472863  1.80284376]
 ...
 [-1.91163459  1.81636357]
 [-1.9123914   1.81062277]
 [-1.91314821  1.80488198]]


In [60]:
# memory days
memory_days = 9

In [61]:
from collections import deque
deq = deque(maxlen=memory_days)

# processing queue--input features
queue_input = []
for i in data_scalar:
    deq.append(list(i))
    if len(deq)==memory_days:
        queue_input.append(list(deq))
print(len(queue_input))      

# processing the final misssing data ( cut the pre_days data )
queue_last = queue_input[-pre_days:]
queue_input = queue_input[:-pre_days]
print(len(queue_input))

queue_output = animal_ID_MAGO_7H_data.iloc[:,2:4].values[memory_days-1:-pre_days]
print(queue_output)

1432
1422
[[-157.678        57.40766667]
 [-157.557        57.56771429]
 [-157.579375     57.54360714]
 ...
 [-157.63185714   57.52871429]
 [-157.64289916   57.47903361]
 [-157.65394118   57.42935294]]


In [62]:
# transfer list to array
queue_input = np.array(queue_input)
g=queue_input.shape
print(g)

queue_output = np.array(queue_output)
print(queue_output.shape)

(1422, 9, 2)
(1422, 2)


In [63]:
def create_trainable_dataset(dataframe,n_inputs,n_outputs):
    X,Y=list(),list()
    for i in range(len(dataframe)-n_inputs-n_outputs+1):
        X.append(dataframe.iloc[i:(i+n_inputs), :])
        Y.append(dataframe.iloc[i + n_inputs:i + n_inputs + n_outputs, 2])
    return np.array(X), np.array(Y)


In [64]:
# split into traning and test datasets
from sklearn.model_selection import train_test_split
queue_input_train,queue_input_test,queue_output_train,queue_output_test = train_test_split(queue_input,queue_output,test_size=0.2)

In [124]:
# def model_builder(hp):
#     model = keras.Sequential()
    

#     # Tune the number of units in the first Dense layer
#     # Choose an optimal value between 32-512
#     hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
#     model.add(LSTM(units=hp_units,input_shape = queue_input.shape[1:]))
#     model.add(Dense(units=hp_units, activation='relu'))
#     model.add(Dense(2))

#     # Tune the learning rate for the optimizer
#     # Choose an optimal value from 0.01, 0.001, or 0.0001
#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                 loss='mse',
#                 metrics=['mape'])
    
#     return model

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import HyperParameters

def build_model(hp):
    drop_out_rate = 0.05
    model = Sequential()
    model.add(layers.LSTM(units=hp.Int('units_input', 32, 64, 32),
                               return_sequences=True,
                               input_shape = (9,2),
                               activation=hp.Choice('activation_input', ['relu','tanh']
                               )))
    
    for i in range(hp.Int('num_layers', 1, 3, 1)):
        model.add(layers.LSTM(units=hp.Int(f"units_{i}", 32, 64, 32),
                              activation=hp.Choice('activation_lstm', ['relu','tanh']), #f"activation_{i}"
                              return_sequences=True))
        model.add(Dropout(drop_out_rate))
        
    model.add(layers.LSTM(units=hp.Int('unit_lstm_last', 32, 64, 32),
                              activation=hp.Choice('activation_last', ['relu', 'tanh'])))
    model.add(Dropout(drop_out_rate))
    
    for i in range(hp.Int('num_layers_dense', 1, 2, 1)):
        model.add(Dense(units=hp.Int(f"units_dense_{i}", 32, 64, 32),
                              activation=hp.Choice(f"activation_dense_{i}", [ 'relu','tanh','sigmoid'])))
        model.add(Dropout(drop_out_rate))
    #model.add(Dense(units=hp.Int('units_dense', 32, 64, 32),activation='relu'))
    #model.add(Dropout(0.1))
    model.add(layers.Dense(2))

    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                  loss='mse',
                  metrics=['mape'])
    return model


In [125]:
from kerastuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_mape',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    overwrite=True,
    project_name='helloworld')


In [126]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [127]:
tuner.search(queue_input_train,queue_output_train, epochs=30, validation_data=(queue_input_test,queue_output_test), callbacks=[stop_early])

Trial 10 Complete [00h 01m 01s]
val_mape: 57.322120666503906

Best val_mape So Far: 3.7345917224884033
Total elapsed time: 00h 08m 33s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [100]:
tuner.search_space_summary()

Search space summary
Default search space size: 8
units_00 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 2, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
units_6 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}


In [128]:
tuner.results_summary()

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_mape", direction="min")

Trial 03 summary
Hyperparameters:
units_input: 32
activation_input: relu
num_layers: 2
units_0: 32
unit_lstm_last: 64
activation_last: relu
num_layers_dense: 2
units_dense_0: 64
activation_dense_0: sigmoid
optimizer: rmsprop
units_1: 64
units_dense_1: 64
activation_dense_1: relu
Score: 3.7345917224884033

Trial 00 summary
Hyperparameters:
units_input: 32
activation_input: relu
num_layers: 1
units_0: 32
unit_lstm_last: 64
activation_last: tanh
num_layers_dense: 1
units_dense_0: 64
activation_dense_0: relu
optimizer: adam
Score: 4.163492202758789

Trial 01 summary
Hyperparameters:
units_input: 32
activation_input: tanh
num_layers: 2
units_0: 64
unit_lstm_last: 64
activation_last: relu
num_layers_dense: 2
units_dense_0: 32
activation_dense_0: tanh
optimizer: rmsprop
units_1: 32
units_dense_1: 32
activation_dense_1: relu
Score: 10.595942497253418

Trial 06 summary
Hyperparameter

In [ ]:
# tuner = kt.Hyperband(model_builder,
#                      objective='val_mape',
#                      max_epochs=10,
#                      factor=3,
#                      directory='my_dir',
#                      project_name='intro_to_kt')

In [103]:
tuner.search(queue_input_train,queue_output_train,epochs=50,validation_data=(queue_input_test,queue_output_test), callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")

INFO:tensorflow:Oracle triggered exit


KeyError: 'units does not exist.'

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(queue_input_train,queue_output_train,epochs=50,validation_data=(queue_input_test,queue_output_test))

val_acc_per_epoch = history.history['val_mape']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(queue_input_train,queue_output_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(queue_input_train,queue_output_train)
print("[test loss, test mape]:", eval_result)